In [6]:
import pandas as pd
import numpy as np
import math
import random
%matplotlib inline
import random
from matplotlib import pyplot as plt
import os
from sklearn import preprocessing

df = pd.read_csv('/Users/eduardorodriguez/Desktop/TFM/DATA/banksim/banksim.csv')
df.head()

le = preprocessing.LabelEncoder()
le.fit(list(set(df['category'])))
df['category'] = le.transform(df['category'])

le.fit(list(set(df['age'])))
df['age'] = le.transform(df['age'])

le.fit(list(set(df['gender'])))
df['gender'] = le.transform(df['gender'])

df_data = df
#X = df.loc[(df.category == 'es_sportsandtoys') | (df.category == 'es_fashion') | (df.category == 'es_travel') | (df.category == 'es_home') |(df.category == 'es_health') | (df.category == 'es_leisure') | df.category == 'es_tech') | (df.category == 'es_otherservices') | (df.category == 'es_barsandrestaurants') |(df.category == 'es_hyper') | (df.category == 'es_wellnessandbeauty')]
y_true = df_data['fraud']

df = df.drop(['customer', 'merchant', 'zipMerchant', 'zipcodeOri'], axis = 1)
df = df.rename(index=str, columns={"fraud": "Class"})
df_data = df_data.drop(['customer', 'merchant', 'zipMerchant', 'zipcodeOri'], axis = 1)

df_data.head()

,step,age,gender,category,amount,fraud
0,0,4,2,12,4.55,0
1,0,2,2,12,39.68,0
2,0,4,1,12,26.89,0
3,0,3,2,12,17.25,0
4,0,5,2,12,35.72,0


In [7]:
ones = df_data[(df_data.Class == 1)]
zeroes = da_data[(df_data.Class == 0)][:len(ones)*4]

X = pd.concat([ones, zeroes])
y_true = X['Class']
df_data = X
df_data.shape

AttributeError: 'DataFrame' object has no attribute 'Class'

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor ## Only available with scikit-learn 0.19 and later
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix,f1_score
X_train, X_test, y_train, y_test = train_test_split(df_data, y_true, test_size=0.3, random_state=42)

In [ ]:
acuracy = []
precision = []
recall = []
f1 = []
timekm = []
for i in [2,3,5,8,10,13,15,17,20]:
    print('Number of Neighbors ' + str(i), end='')
    print('\\\\')
    print('\\\\')
    clf=LocalOutlierFactor(n_neighbors=i, algorithm='auto', leaf_size=30, \
                   metric='minkowski', p=2, metric_params=None, contamination=0.1, n_jobs=-1)
    %%timeit
    clf.fit(X_test)
    clf.fit(X_test)
    %%timeit
    lof_y_pred=clf.negative_outlier_factor_

    #Creating class labels based on decision function
    lof_y_pred_class=lof_y_pred.copy()
    lof_y_pred_class[lof_y_pred>=np.percentile(lof_y_pred,95)]=1
    lof_y_pred_class[lof_y_pred<np.percentile(lof_y_pred,95)]=0
    
    lof_y_pred=clf.negative_outlier_factor_

    #Creating class labels based on decision function
    lof_y_pred_class=lof_y_pred.copy()
    lof_y_pred_class[lof_y_pred>=np.percentile(lof_y_pred,95)]=1
    lof_y_pred_class[lof_y_pred<np.percentile(lof_y_pred,95)]=0
    
    roc_auc_score(y_test, lof_y_pred_class)
    
    knn_accuracy_score=metrics.accuracy_score(y_test, lof_y_pred_class)
    knn_precison_score=metrics.precision_score(y_test, lof_y_pred_class)
    knn_recall_score=metrics.recall_score(y_test, lof_y_pred_class)
    knn_f1_score=metrics.f1_score(y_test, lof_y_pred_class)
    
    precision, recall, _ = metrics.precision_recall_curve(y_test, lof_y_pred_class)
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(metrics.average_precision_score(y_test, lof_y_pred_class)))
    
    
    km_cm=confusion_matrix(y_test, lof_y_pred_class)
    print("LOF Local Outlier Factor\\\\")
    print("Confusion Matrix\\\\")
    print("tn =",km_cm[0][0],"fp =",km_cm[0][1], end='')
    print('\\\\')
    print("fn =",km_cm[1][0],"tp =",km_cm[1][1], end='')
    print('\\\\')
    print("Scores", end='')
    print('\\\\')
    print("Accuracy -->",knn_accuracy_score, end='')
    print('\\\\')
    print("Precison -->",knn_precison_score, end='')
    print('\\\\')
    print("Recall -->",knn_recall_score, end='')
    print('\\\\')
    print("F1 -->",knn_f1_score, end='')
    print('\\\\')
    print('\\\\')
    print('\\\\')
    
    acuracy += [knn_accuracy_score,]
    precision += [knn_precison_score,]
    recall += [knn_recall_score,]
    f1 += [knn_f1_score,]

In [ ]:
pyplot.title('Acuracy Scores')
pyplot.barh(['2','3','5','8','10','13','15','17','20'], list(map(lambda x: x - min(acuracy) + 0.001, acuracy)), align='center')
pyplot.xlabel('Acuracy Scores')

In [ ]:
pyplot.title('Precision Scores')
pyplot.barh(['2','3','5','8','10','13','15','17','20'], list(map(lambda x: x - min(precision) + 0.01, precision)), align='center')
pyplot.xlabel('Precision Scores')

In [ ]:
pyplot.title('Recall Scores')
pyplot.barh(['2','3','5','8','10','13','15','17','20'], list(map(lambda x: x - min(recall) + 0.01, recall)), align='center')
pyplot.xlabel('Recall Scores')

In [ ]:
pyplot.title('F1 Scores')
pyplot.barh(['2','3','5','8','10','13','15','17','20'], list(map(lambda x: x - min(f1) + 0.01, f1)), align='center')
pyplot.xlabel('F1 Scores')

In [ ]:
from matplotlib import pyplot as plt
def get_predictions(lof, threshold=1.5):
    return list(map(lambda x: -1 if x > threshold else 1, lof))

lof_ths = np.arange(0.1, 1, 0.1)
clf=LocalOutlierFactor(n_neighbors=8, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, contamination=0.1, n_jobs=-1)
clf.fit_predict(df.drop(['Class'], axis=1))
lofs = -clf.negative_outlier_factor_

plt.figure()
lw = 2
plt.xlim([0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver operating characteristic', fontsize=16)

fpr = tpr = [0]

for ths in lof_ths:
    pred = get_predictions(lofs, threshold=ths)
    tn, fp, fn, tp = confusion_matrix(df.Class, pred, labels=[-1,1]).ravel()
    fpr.append(fp / (fp + tn + .000001))
    tpr.append(tp / (tp + fn + .000001))

fpr.append(1)
tpr.append(1)
plt.plot(fpr, tpr, label='ROC Curve')
plt.plot([0, 1], [0, 1], color='navy', label='Random Guessing', lw=lw, linestyle='--')
plt.legend(loc="lower right", fontsize=12)

In [ ]:
[
    FastABOD(novelty=True, 
             n_jobs=-1
            ),
    MiniBatchKMeans(13,
                    "kd_tree"
                   ), 
    LOF(n_neighbors=8, 
        algorithm='auto', 
        leaf_size=30, 
        metric='minkowski',
        p=2, 
        metric_params=None, 
        contamination=0.1, 
        n_jobs=-1
       ),
    KNN(novelty=True, 
        n_jobs=-1
       ), 
    IForest(n_estimators=100, 
            max_samples='auto', 
            contamination=0.01,
            max_features=1.0, 
            bootstrap=False, 
            n_jobs=-1, 
            random_state=42
           ),
    PCA(), 
    KDE()
]